In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
import numpy as np
from torch.optim.lr_scheduler import StepLR
import copy
from sklearn.neighbors import KNeighborsClassifier

feature_dir = '/kaggle/working/files'
os.makedirs(feature_dir, exist_ok=True)

# Distillation Loss Function
def distillation_loss(student_outputs, teacher_outputs, temperature=2.0):
    teacher_probs = F.softmax(teacher_outputs / temperature, dim=1)
    student_probs = F.log_softmax(student_outputs / temperature, dim=1)
    return F.kl_div(student_probs, teacher_probs, reduction='batchmean') * (temperature ** 2)

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, images, targets=None, transform=None):
        self.images = images
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        target = self.targets[idx] if self.targets is not None else -1
        return image, target

    
# Load Data from .pth file
def load_data_from_pth(pth_path, has_targets=True):
    data_dict = torch.load(pth_path)
    data = data_dict['data']
    targets = data_dict['targets'] if has_targets else None
    return data, targets

# Paths to Datasets
# Edit these paths to run the code on your machine
d1_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/1_train_data.tar.pth'
d1hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth'
d2_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/2_train_data.tar.pth'
d2hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth'
d3_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/3_train_data.tar.pth'
d3hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth'

d4_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/4_train_data.tar.pth'
d4hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/4_eval_data.tar.pth'

d5_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/5_train_data.tar.pth'
d5hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/5_eval_data.tar.pth'

d6_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/6_train_data.tar.pth'
d6hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/6_eval_data.tar.pth'

d7_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/7_train_data.tar.pth'
d7hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/7_eval_data.tar.pth'

d8_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/8_train_data.tar.pth'
d8hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/8_eval_data.tar.pth'

d9_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/9_train_data.tar.pth'
d9hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/9_eval_data.tar.pth'

d10_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/train_data/10_train_data.tar.pth'
d10hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_one_dataset/eval_data/10_eval_data.tar.pth'

d11_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/1_train_data.tar.pth'
d11hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/1_eval_data.tar.pth'

d12_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/2_train_data.tar.pth'
d12hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/2_eval_data.tar.pth'

d13_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/3_train_data.tar.pth'
d13hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/3_eval_data.tar.pth'

d14_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/4_train_data.tar.pth'
d14hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/4_eval_data.tar.pth'

d15_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/5_train_data.tar.pth'
d15hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/5_eval_data.tar.pth'

d16_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/6_train_data.tar.pth'
d16hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/6_eval_data.tar.pth'

d17_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/7_train_data.tar.pth'
d17hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/7_eval_data.tar.pth'

d18_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/8_train_data.tar.pth'
d18hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/8_eval_data.tar.pth'

d19_path = '//kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/9_train_data.tar.pth'
d19hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/9_eval_data.tar.pth'

d20_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/train_data/10_train_data.tar.pth'
d20hat_path = '/kaggle/input/dataset-for-proj-2/dataset/dataset/part_two_dataset/eval_data/10_eval_data.tar.pth'

print({"reached_path_to_datasets"})

# Load Datasets
d1_data, d1_targets = load_data_from_pth(d1_path)
d1hat_data, d1hat_targets = load_data_from_pth(d1hat_path)
d2_data, _ = load_data_from_pth(d2_path, has_targets=False)
d2hat_data, d2hat_targets = load_data_from_pth(d2hat_path)

d3_data, _ = load_data_from_pth(d3_path, has_targets=False)
d3hat_data, d3hat_targets = load_data_from_pth(d3hat_path)

d4_data, _ = load_data_from_pth(d4_path, has_targets=False)
d4hat_data, d4hat_targets = load_data_from_pth(d4hat_path)

d5_data, _ = load_data_from_pth(d5_path, has_targets=False)
d5hat_data, d5hat_targets = load_data_from_pth(d5hat_path)

d6_data, _ = load_data_from_pth(d6_path, has_targets=False)
d6hat_data, d6hat_targets = load_data_from_pth(d6hat_path)

d7_data, _ = load_data_from_pth(d7_path, has_targets=False)
d7hat_data, d7hat_targets = load_data_from_pth(d7hat_path)

d8_data, _ = load_data_from_pth(d8_path, has_targets=False)
d8hat_data, d8hat_targets = load_data_from_pth(d8hat_path)

d9_data, _ = load_data_from_pth(d9_path, has_targets=False)
d9hat_data, d9hat_targets = load_data_from_pth(d9hat_path)

d10_data, _ = load_data_from_pth(d10_path, has_targets=False)
d10hat_data, d10hat_targets = load_data_from_pth(d10hat_path)

d11_data, _ = load_data_from_pth(d11_path, has_targets=False)
d11hat_data, d11hat_targets = load_data_from_pth(d11hat_path)

d12_data, _ = load_data_from_pth(d12_path, has_targets=False)
d12hat_data, d12hat_targets = load_data_from_pth(d12hat_path)

d13_data, _ = load_data_from_pth(d13_path, has_targets=False)
d13hat_data, d13hat_targets = load_data_from_pth(d13hat_path)

d14_data, _ = load_data_from_pth(d14_path, has_targets=False)
d14hat_data, d14hat_targets = load_data_from_pth(d14hat_path)

d15_data, _ = load_data_from_pth(d15_path, has_targets=False)
d15hat_data, d15hat_targets = load_data_from_pth(d15hat_path)

d16_data, _ = load_data_from_pth(d16_path, has_targets=False)
d16hat_data, d16hat_targets = load_data_from_pth(d16hat_path)

d17_data, _ = load_data_from_pth(d17_path, has_targets=False)
d17hat_data, d17hat_targets = load_data_from_pth(d17hat_path)

d18_data, _ = load_data_from_pth(d18_path, has_targets=False)
d18hat_data, d18hat_targets = load_data_from_pth(d18hat_path)

d19_data, _ = load_data_from_pth(d19_path, has_targets=False)
d19hat_data, d19hat_targets = load_data_from_pth(d19hat_path)

d20_data, _ = load_data_from_pth(d20_path, has_targets=False)
d20hat_data, d20hat_targets = load_data_from_pth(d20hat_path)


# Data Transformations
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# DataLoaders
train_dataset = CustomImageDataset(d1_data, d1_targets, transform=train_transform)
d1hat_dataset = CustomImageDataset(d1hat_data, d1hat_targets, transform=test_transform)
d2hat_dataset = CustomImageDataset(d2hat_data, d2hat_targets, transform=test_transform)
d3hat_dataset = CustomImageDataset(d3hat_data, d3hat_targets, transform=test_transform)
d4hat_dataset = CustomImageDataset(d4hat_data, d4hat_targets, transform=test_transform)
d5hat_dataset = CustomImageDataset(d5hat_data, d5hat_targets, transform=test_transform)
d6hat_dataset = CustomImageDataset(d6hat_data, d6hat_targets, transform=test_transform)
d7hat_dataset = CustomImageDataset(d7hat_data, d7hat_targets, transform=test_transform)
d8hat_dataset = CustomImageDataset(d8hat_data, d8hat_targets, transform=test_transform)
d9hat_dataset = CustomImageDataset(d9hat_data, d9hat_targets, transform=test_transform)
d10hat_dataset = CustomImageDataset(d10hat_data, d10hat_targets, transform=test_transform)
d11hat_dataset = CustomImageDataset(d11hat_data, d11hat_targets, transform=test_transform)
d12hat_dataset = CustomImageDataset(d12hat_data, d12hat_targets, transform=test_transform)
d13hat_dataset = CustomImageDataset(d13hat_data, d13hat_targets, transform=test_transform)

# For 14
d14hat_dataset = CustomImageDataset(d14hat_data, d14hat_targets, transform=test_transform)
# For 15
d15hat_dataset = CustomImageDataset(d15hat_data, d15hat_targets, transform=test_transform)
# For 16
d16hat_dataset = CustomImageDataset(d16hat_data, d16hat_targets, transform=test_transform)
# For 17
d17hat_dataset = CustomImageDataset(d17hat_data, d17hat_targets, transform=test_transform)
# For 18
d18hat_dataset = CustomImageDataset(d18hat_data, d18hat_targets, transform=test_transform)
# For 19
d19hat_dataset = CustomImageDataset(d19hat_data, d19hat_targets, transform=test_transform)
# For 20
d20hat_dataset = CustomImageDataset(d20hat_data, d20hat_targets, transform=test_transform)



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
d1hat_loader = DataLoader(d1hat_dataset, batch_size=32, shuffle=False)
d2hat_loader = DataLoader(d2hat_dataset, batch_size=32, shuffle=False)
d3hat_loader = DataLoader(d3hat_dataset, batch_size=32, shuffle=False)

d4hat_loader = DataLoader(d4hat_dataset, batch_size=32, shuffle=False)
d5hat_loader = DataLoader(d5hat_dataset, batch_size=32, shuffle=False)
d6hat_loader = DataLoader(d6hat_dataset, batch_size=32, shuffle=False)
d7hat_loader = DataLoader(d7hat_dataset, batch_size=32, shuffle=False)
d8hat_loader = DataLoader(d8hat_dataset, batch_size=32, shuffle=False)
d9hat_loader = DataLoader(d9hat_dataset, batch_size=32, shuffle=False)
d10hat_loader = DataLoader(d10hat_dataset, batch_size=32, shuffle=False)
d11hat_loader = DataLoader(d11hat_dataset, batch_size=32, shuffle=False)
d12hat_loader = DataLoader(d12hat_dataset, batch_size=32, shuffle=False)
d13hat_loader = DataLoader(d13hat_dataset, batch_size=32, shuffle=False)

# For 14
d14hat_loader = DataLoader(d14hat_dataset, batch_size=32, shuffle=False)
# For 15
d15hat_loader = DataLoader(d15hat_dataset, batch_size=32, shuffle=False)
# For 16
d16hat_loader = DataLoader(d16hat_dataset, batch_size=32, shuffle=False)
# For 17
d17hat_loader = DataLoader(d17hat_dataset, batch_size=32, shuffle=False)
# For 18
d18hat_loader = DataLoader(d18hat_dataset, batch_size=32, shuffle=False)
# For 19
d19hat_loader = DataLoader(d19hat_dataset, batch_size=32, shuffle=False)
# For 20
d20hat_loader = DataLoader(d20hat_dataset, batch_size=32, shuffle=False)


# Feature Extractor: ResNet50
class ResNet50Extractor(nn.Module):
    def __init__(self):
        super(ResNet50Extractor, self).__init__()
        resnet_model = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet_model.children())[:-1])

    def forward(self, x):
        return self.features(x)
    
# Learning with Prototypes Model
class LearningWithPrototypes(nn.Module):
    def __init__(self, feature_extractor, num_classes, prototype_dim=2048):
        super(LearningWithPrototypes, self).__init__()
        self.feature_extractor = feature_extractor
        self.fc = nn.Linear(prototype_dim, num_classes)
        self.num_classes = num_classes
        self.prototypes = nn.Parameter(torch.randn(num_classes, prototype_dim))

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        logits = self.fc(features)
        return logits, features

    def prototype_loss(self, features, labels):
        distances = torch.cdist(features, self.prototypes)
        return F.cross_entropy(-distances, labels)
    
    def update_prototypes(self, features, labels):
        for class_idx in range(self.num_classes):
            class_features = features[labels == class_idx]
            if class_features.size(0) > 0:
                self.prototypes.data[class_idx] = class_features.mean(dim=0)

    def update_prototypes_with_ratp(self, features, labels, alpha=0.5):
        """
        Update prototypes with new high-confidence samples.

        Parameters:
        - features: Tensor of shape (batch_size, feature_dim), feature representations of the dataset.
        - labels: Tensor of shape (batch_size,), corresponding pseudo-labels.
        - alpha: Weight for blending old and new prototypes (default: 0.5).
        """
        for class_idx in range(self.num_classes):
            # Select features belonging to the current class
            class_mask = (labels == class_idx)
            class_features = features[class_mask]  # Shape: (num_samples_in_class, feature_dim)
            
            if class_features.size(0) > 0:
                # Compute new centroid for this class (average over batch dimension)
                new_centroid = class_features.mean(dim=0)  # Shape: (feature_dim,)
                
                # Blend the old and new prototypes
                self.prototypes.data[class_idx] = (
                    alpha * self.prototypes.data[class_idx] + (1 - alpha) * new_centroid
                )
                
                
                
# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor = ResNet50Extractor().to(device)
model = LearningWithPrototypes(feature_extractor, num_classes=len(set(d1_targets))).to(device)


print({"started"})

# Load Pretrained Prototypes if Available
if os.path.exists('/kaggle/input/weight-files/prototypes.pth'):
    model.prototypes.data = torch.load('/kaggle/input/weight-files/prototypes.pth')
    print("Loaded saved prototypes.")

# Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
alpha = 0.1
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


def self_train_update_with_t2pl(model, unlabeled_data, num, num_epochs=15, temperature=2.0, alpha=0.5, beta=0.7, top_percentage=0.3, top_5_percentage=0.3):
    model.eval()
    
    # Step 1: Generate softmax outputs and select the top confident samples
    softmax_outputs = []
    representations = []
    
    with torch.no_grad():
        for images, _ in DataLoader(CustomImageDataset(unlabeled_data, transform=test_transform), batch_size=32):
            images = images.to(device)
            outputs, features = model(images)
            softmax_probs = F.softmax(outputs / temperature, dim=1)
            softmax_outputs.append(softmax_probs.cpu().numpy())
            representations.append(features.cpu().numpy())
    
    # Concatenate all softmax outputs and features
    softmax_outputs = np.concatenate(softmax_outputs, axis=0)
    representations = np.concatenate(representations, axis=0)
    
    # Step 2: Select top confident samples based on a percentage of the data
    top_k_count = int(top_percentage * softmax_outputs.shape[0])  # Calculate the top percentage dynamically
    top_k_indices = np.argsort(np.max(softmax_outputs, axis=1))[-top_k_count:]
    top_k_softmax = softmax_outputs[top_k_indices]
    top_k_representations = representations[top_k_indices]
    
    # Step 3: Construct class centroids using the top samples
    class_centroids = []
    num_classes = softmax_outputs.shape[1]
    
    for class_idx in range(num_classes):
        class_samples = top_k_representations[np.argmax(top_k_softmax, axis=1) == class_idx]
        if class_samples.shape[0] > 0:
            centroid = class_samples.mean(axis=0)
        else:
            centroid = np.zeros_like(top_k_representations[0])
        class_centroids.append(centroid)
    
    class_centroids = np.array(class_centroids)
    
    # Step 4: Calculate cosine similarity between centroids and representations of all unlabeled samples
    similarities = np.dot(representations, class_centroids.T) / (np.linalg.norm(representations, axis=1)[:, None] * np.linalg.norm(class_centroids, axis=1))
    
    # Step 5: Select top half of the samples for pseudo-labeling
    top_half_count = representations.shape[0] // 2  # Calculate the top 50% based on the number of samples
    top_half_indices = np.argsort(np.max(similarities, axis=1))[-top_half_count:]
    
    # Step 6: Use kNN for final pseudo-label assignment
    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(top_k_representations, np.argmax(top_k_softmax, axis=1))
    final_pseudo_labels = knn.predict(representations[top_half_indices])
    
    # Step 7: Select top 5% samples based on Euclidean distance
    distances = knn.kneighbors(representations[top_half_indices], return_distance=True)[0]
    top_5_count = int(top_5_percentage * top_half_count)
    top_5_indices = np.argsort(distances.min(axis=1))[:top_5_count]
    
    # Filter `unlabeled_data` to include only samples in `top_5_indices`
    filtered_unlabeled_data = [unlabeled_data[top_half_indices[idx]] for idx in top_5_indices]
    
    # Update pseudo dataset with new pseudo labels, only for the selected samples
    pseudo_dataset = CustomImageDataset(filtered_unlabeled_data, final_pseudo_labels[top_5_indices], transform=train_transform)
    pseudo_loader = DataLoader(pseudo_dataset, batch_size=32, shuffle=True)

    # Fine-tune model on the pseudo-labeled data
    new_model = copy.deepcopy(model)
    optimizer = optim.AdamW(new_model.parameters(), lr=0.00001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    
    # To track loss
    epoch_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        running_loss = 0.0
        new_model.train()
        for images, labels in pseudo_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, features = new_model(images)
            with torch.no_grad():
                teacher_outputs, _ = model(images)
            ce_loss = criterion(outputs, labels)
            proto_loss = new_model.prototype_loss(features, labels)
            dist_loss = distillation_loss(outputs, teacher_outputs, temperature)
            loss = ce_loss + alpha * proto_loss + beta * dist_loss
            print(f"Epoch {epoch} has a loss of {loss}")
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        
        # Average loss over the epoch
        epoch_loss = running_loss / len(pseudo_loader.dataset)
        epoch_losses.append(epoch_loss)
        print(f"Epoch {epoch+1}, Loss: {epoch_loss}")

        scheduler.step()

         # Step 3: Update Prototypes with high-confidence samples
        with torch.no_grad():
            all_features = []
            all_labels = []
            for images, labels in pseudo_loader:
                images = images.to(device)
                
                # Extract features directly from feature extractor
                features = new_model.feature_extractor(images).view(images.size(0), -1)
                all_features.append(features)
                all_labels.append(labels.to(device))
            
            all_features = torch.cat(all_features, dim=0)
            all_labels = torch.cat(all_labels, dim=0)
            
            # Update prototypes with high-confidence samples
            new_model.update_prototypes_with_ratp(all_features, all_labels, alpha=0.5)
    with torch.no_grad():
        weight_old = (num - 1) / num
        weight_new = 1 / num
        new_model.prototypes.data = (
            weight_old * model.prototypes.data + weight_new * new_model.prototypes.data
        )
        print("Prototypes updated using weighted average.")
    # Save model after self-training
    torch.save(new_model.state_dict(), f'/kaggle/working/files/model_f{num}_weights.pth')
    print("Saved model after self-training with T2PL.")
    
 
    
    return new_model




# Function to Calculate Accuracy
def calculate_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs, _ = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total


accuracy_matrix = []

if os.path.exists('/kaggle/input/d10-weights/model_f10_weights.pth'):
    model_f10 = copy.deepcopy(model)
    model_f10.load_state_dict(torch.load('/kaggle/input/d10-weights/model_f10_weights.pth'))
    print("Loaded self-trained model. for model 10")
else:
    print("Model after training on dataset 10 is not availabale, please load it with the appropriate address,")





# Train on Unlabeled Data (D11) using self-training and distillation
if os.path.exists('/kaggle/working/files/model_f11_weights.pth'):
    model_f11 = copy.deepcopy(model)
    model_f11.load_state_dict(torch.load('/kaggle/working/files/model_f11_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f11 = self_train_update_with_t2pl(model_f10, d11_data, 11)

    
print("started_accuracy_computation")
f11_acc_d1hat = calculate_accuracy(model_f11,d1hat_loader)
print({f11_acc_d1hat})
f11_acc_d2hat = calculate_accuracy(model_f11,d2hat_loader)
print({f11_acc_d2hat})
f11_acc_d3hat = calculate_accuracy(model_f11,d3hat_loader)
print({f11_acc_d3hat})
f11_acc_d4hat = calculate_accuracy(model_f11,d4hat_loader)
print({f11_acc_d4hat})
f11_acc_d5hat = calculate_accuracy(model_f11,d5hat_loader)
print({f11_acc_d5hat})
f11_acc_d6hat = calculate_accuracy(model_f11,d6hat_loader)
print({f11_acc_d6hat})
f11_acc_d7hat = calculate_accuracy(model_f11,d7hat_loader)
print({f11_acc_d7hat})
f11_acc_d8hat = calculate_accuracy(model_f11,d8hat_loader)
print({f11_acc_d8hat})
f11_acc_d9hat = calculate_accuracy(model_f11,d9hat_loader)
print({f11_acc_d9hat})
f11_acc_d10hat = calculate_accuracy(model_f11,d10hat_loader)
print({f11_acc_d10hat})

f11_acc_d11hat = calculate_accuracy(model_f11,d11hat_loader)
print({f11_acc_d11hat})
accuracy_matrix.append([f11_acc_d1hat,f11_acc_d2hat,f11_acc_d3hat,f11_acc_d4hat,f11_acc_d5hat,f11_acc_d6hat,f11_acc_d7hat,f11_acc_d8hat,f11_acc_d9hat,f11_acc_d10hat,f11_acc_d11hat,None,None,None,None,None,None,None,None,None])


# Train on Unlabeled Data (D12) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f12_weights.pth'):
    model_f12 = copy.deepcopy(model)
    model_f12.load_state_dict(torch.load('/kaggle/input/weight-files/model_f12_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f12 = self_train_update_with_t2pl(model_f11, d12_data,12)
    
print("started_accuracy_computation")
f12_acc_d1hat = calculate_accuracy(model_f12,d1hat_loader)
print({f12_acc_d1hat})
f12_acc_d2hat = calculate_accuracy(model_f12,d2hat_loader)
print({f12_acc_d2hat})
f12_acc_d3hat = calculate_accuracy(model_f12,d3hat_loader)
print({f12_acc_d3hat})
f12_acc_d4hat = calculate_accuracy(model_f12,d4hat_loader)
print({f12_acc_d4hat})
f12_acc_d5hat = calculate_accuracy(model_f12,d5hat_loader)
print({f12_acc_d5hat})
f12_acc_d6hat = calculate_accuracy(model_f12,d6hat_loader)
print({f12_acc_d6hat})
f12_acc_d7hat = calculate_accuracy(model_f12,d7hat_loader)
print({f12_acc_d7hat})
f12_acc_d8hat = calculate_accuracy(model_f12,d8hat_loader)
print({f12_acc_d8hat})
f12_acc_d9hat = calculate_accuracy(model_f12,d9hat_loader)
print({f12_acc_d9hat})
f12_acc_d10hat = calculate_accuracy(model_f12,d10hat_loader)
print({f12_acc_d10hat})
f12_acc_d11hat = calculate_accuracy(model_f12,d11hat_loader)
print({f12_acc_d11hat})
f12_acc_d12hat = calculate_accuracy(model_f12,d12hat_loader)
print({f12_acc_d12hat})
accuracy_matrix.append([f12_acc_d1hat,f12_acc_d2hat,f12_acc_d3hat,f12_acc_d4hat,f12_acc_d5hat,f12_acc_d6hat,f12_acc_d7hat,f12_acc_d8hat,f12_acc_d9hat,f12_acc_d10hat,f12_acc_d11hat,f12_acc_d12hat,None,None,None,None,None,None,None,None])




# Train on Unlabeled Data (D13) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f13_weights.pth'):
    model_f13 = copy.deepcopy(model)
    model_f13.load_state_dict(torch.load('/kaggle/input/weight-files/model_f13_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f13 = self_train_update_with_t2pl(model_f12, d13_data, 13)
    
print("started_accuracy_computation")
f13_acc_d1hat = calculate_accuracy(model_f13, d1hat_loader)
print({f13_acc_d1hat})
f13_acc_d2hat = calculate_accuracy(model_f13, d2hat_loader)
print({f13_acc_d2hat})
f13_acc_d3hat = calculate_accuracy(model_f13, d3hat_loader)
print({f13_acc_d3hat})
f13_acc_d4hat = calculate_accuracy(model_f13, d4hat_loader)
print({f13_acc_d4hat})
f13_acc_d5hat = calculate_accuracy(model_f13, d5hat_loader)
print({f13_acc_d5hat})
f13_acc_d6hat = calculate_accuracy(model_f13, d6hat_loader)
print({f13_acc_d6hat})
f13_acc_d7hat = calculate_accuracy(model_f13, d7hat_loader)
print({f13_acc_d7hat})
f13_acc_d8hat = calculate_accuracy(model_f13, d8hat_loader)
print({f13_acc_d8hat})
f13_acc_d9hat = calculate_accuracy(model_f13, d9hat_loader)
print({f13_acc_d9hat})
f13_acc_d10hat = calculate_accuracy(model_f13, d10hat_loader)
print({f13_acc_d10hat})
f13_acc_d11hat = calculate_accuracy(model_f13, d11hat_loader)
print({f13_acc_d11hat})
f13_acc_d12hat = calculate_accuracy(model_f13, d12hat_loader)
print({f13_acc_d12hat})
f13_acc_d13hat = calculate_accuracy(model_f13, d13hat_loader)
print({f13_acc_d13hat})
accuracy_matrix.append([f13_acc_d1hat, f13_acc_d2hat, f13_acc_d3hat, f13_acc_d4hat, f13_acc_d5hat, f13_acc_d6hat, f13_acc_d7hat, f13_acc_d8hat, f13_acc_d9hat, f13_acc_d10hat, f13_acc_d11hat, f13_acc_d12hat, f13_acc_d13hat,None,None,None,None,None,None,None])






# Train on Unlabeled Data (D14) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f14_weights.pth'):
    model_f14 = copy.deepcopy(model)
    model_f14.load_state_dict(torch.load('/kaggle/input/weight-files/model_f14_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f14 = self_train_update_with_t2pl(model_f13, d14_data, 14)
    
print("started_accuracy_computation")
f14_acc_d1hat = calculate_accuracy(model_f14, d1hat_loader)
print({f14_acc_d1hat})
f14_acc_d2hat = calculate_accuracy(model_f14, d2hat_loader)
print({f14_acc_d2hat})
f14_acc_d3hat = calculate_accuracy(model_f14, d3hat_loader)
print({f14_acc_d3hat})
f14_acc_d4hat = calculate_accuracy(model_f14, d4hat_loader)
print({f14_acc_d4hat})
f14_acc_d5hat = calculate_accuracy(model_f14, d5hat_loader)
print({f14_acc_d5hat})
f14_acc_d6hat = calculate_accuracy(model_f14, d6hat_loader)
print({f14_acc_d6hat})
f14_acc_d7hat = calculate_accuracy(model_f14, d7hat_loader)
print({f14_acc_d7hat})
f14_acc_d8hat = calculate_accuracy(model_f14, d8hat_loader)
print({f14_acc_d8hat})
f14_acc_d9hat = calculate_accuracy(model_f14, d9hat_loader)
print({f14_acc_d9hat})
f14_acc_d10hat = calculate_accuracy(model_f14, d10hat_loader)
print({f14_acc_d10hat})
f14_acc_d11hat = calculate_accuracy(model_f14, d11hat_loader)
print({f14_acc_d11hat})
f14_acc_d12hat = calculate_accuracy(model_f14, d12hat_loader)
print({f14_acc_d12hat})
f14_acc_d13hat = calculate_accuracy(model_f14, d13hat_loader)
print({f14_acc_d13hat})
f14_acc_d14hat = calculate_accuracy(model_f14, d14hat_loader)
print({f14_acc_d14hat})
accuracy_matrix.append([f14_acc_d1hat, f14_acc_d2hat, f14_acc_d3hat, f14_acc_d4hat, f14_acc_d5hat, f14_acc_d6hat, f14_acc_d7hat, f14_acc_d8hat, f14_acc_d9hat, f14_acc_d10hat, f14_acc_d11hat, f14_acc_d12hat, f14_acc_d13hat, f14_acc_d14hat,None,None,None,None,None,None])



# Train on Unlabeled Data (D15) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f15_weights.pth'):
    model_f15 = copy.deepcopy(model)
    model_f15.load_state_dict(torch.load('/kaggle/input/weight-files/model_f15_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f15 = self_train_update_with_t2pl(model_f14, d15_data, 15)
    
print("started_accuracy_computation")
f15_acc_d1hat = calculate_accuracy(model_f15, d1hat_loader)
print({f15_acc_d1hat})
f15_acc_d2hat = calculate_accuracy(model_f15, d2hat_loader)
print({f15_acc_d2hat})
f15_acc_d3hat = calculate_accuracy(model_f15, d3hat_loader)
print({f15_acc_d3hat})
f15_acc_d4hat = calculate_accuracy(model_f15, d4hat_loader)
print({f15_acc_d4hat})
f15_acc_d5hat = calculate_accuracy(model_f15, d5hat_loader)
print({f15_acc_d5hat})
f15_acc_d6hat = calculate_accuracy(model_f15, d6hat_loader)
print({f15_acc_d6hat})
f15_acc_d7hat = calculate_accuracy(model_f15, d7hat_loader)
print({f15_acc_d7hat})
f15_acc_d8hat = calculate_accuracy(model_f15, d8hat_loader)
print({f15_acc_d8hat})
f15_acc_d9hat = calculate_accuracy(model_f15, d9hat_loader)
print({f15_acc_d9hat})
f15_acc_d10hat = calculate_accuracy(model_f15, d10hat_loader)
print({f15_acc_d10hat})
f15_acc_d11hat = calculate_accuracy(model_f15, d11hat_loader)
print({f15_acc_d11hat})
f15_acc_d12hat = calculate_accuracy(model_f15, d12hat_loader)
print({f15_acc_d12hat})
f15_acc_d13hat = calculate_accuracy(model_f15, d13hat_loader)
print({f15_acc_d13hat})
f15_acc_d14hat = calculate_accuracy(model_f15, d14hat_loader)
print({f15_acc_d14hat})
f15_acc_d15hat = calculate_accuracy(model_f15, d15hat_loader)
print({f15_acc_d15hat})
accuracy_matrix.append([f15_acc_d1hat, f15_acc_d2hat, f15_acc_d3hat, f15_acc_d4hat, f15_acc_d5hat, f15_acc_d6hat, f15_acc_d7hat, f15_acc_d8hat, f15_acc_d9hat, f15_acc_d10hat, f15_acc_d11hat, f15_acc_d12hat, f15_acc_d13hat, f15_acc_d14hat, f15_acc_d15hat,None,None,None,None,None])



# Train on Unlabeled Data (D16) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f16_weights.pth'):
    model_f16 = copy.deepcopy(model)
    model_f16.load_state_dict(torch.load('/kaggle/input/weight-files/model_f16_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f16 = self_train_update_with_t2pl(model_f15, d16_data, 16)
    
print("started_accuracy_computation")
f16_acc_d1hat = calculate_accuracy(model_f16, d1hat_loader)
print({f16_acc_d1hat})
f16_acc_d2hat = calculate_accuracy(model_f16, d2hat_loader)
print({f16_acc_d2hat})
f16_acc_d3hat = calculate_accuracy(model_f16, d3hat_loader)
print({f16_acc_d3hat})
f16_acc_d4hat = calculate_accuracy(model_f16, d4hat_loader)
print({f16_acc_d4hat})
f16_acc_d5hat = calculate_accuracy(model_f16, d5hat_loader)
print({f16_acc_d5hat})
f16_acc_d6hat = calculate_accuracy(model_f16, d6hat_loader)
print({f16_acc_d6hat})
f16_acc_d7hat = calculate_accuracy(model_f16, d7hat_loader)
print({f16_acc_d7hat})
f16_acc_d8hat = calculate_accuracy(model_f16, d8hat_loader)
print({f16_acc_d8hat})
f16_acc_d9hat = calculate_accuracy(model_f16, d9hat_loader)
print({f16_acc_d9hat})
f16_acc_d10hat = calculate_accuracy(model_f16, d10hat_loader)
print({f16_acc_d10hat})
f16_acc_d11hat = calculate_accuracy(model_f16, d11hat_loader)
print({f16_acc_d11hat})
f16_acc_d12hat = calculate_accuracy(model_f16, d12hat_loader)
print({f16_acc_d12hat})
f16_acc_d13hat = calculate_accuracy(model_f16, d13hat_loader)
print({f16_acc_d13hat})
f16_acc_d14hat = calculate_accuracy(model_f16, d14hat_loader)
print({f16_acc_d14hat})
f16_acc_d15hat = calculate_accuracy(model_f16, d15hat_loader)
print({f16_acc_d15hat})
f16_acc_d16hat = calculate_accuracy(model_f16, d16hat_loader)
print({f16_acc_d16hat})
accuracy_matrix.append([f16_acc_d1hat, f16_acc_d2hat, f16_acc_d3hat, f16_acc_d4hat, f16_acc_d5hat, f16_acc_d6hat, f16_acc_d7hat, f16_acc_d8hat, f16_acc_d9hat, f16_acc_d10hat, f16_acc_d11hat, f16_acc_d12hat, f16_acc_d13hat, f16_acc_d14hat, f16_acc_d15hat, f16_acc_d16hat,None,None,None,None])



# Train on Unlabeled Data (D17) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f17_weights.pth'):
    model_f17 = copy.deepcopy(model)
    model_f17.load_state_dict(torch.load('/kaggle/input/weight-files/model_f17_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f17 = self_train_update_with_t2pl(model_f16, d17_data, 17)
    
print("started_accuracy_computation")
f17_acc_d1hat = calculate_accuracy(model_f17, d1hat_loader)
print({f17_acc_d1hat})
f17_acc_d2hat = calculate_accuracy(model_f17, d2hat_loader)
print({f17_acc_d2hat})
f17_acc_d3hat = calculate_accuracy(model_f17, d3hat_loader)
print({f17_acc_d3hat})
f17_acc_d4hat = calculate_accuracy(model_f17, d4hat_loader)
print({f17_acc_d4hat})
f17_acc_d5hat = calculate_accuracy(model_f17, d5hat_loader)
print({f17_acc_d5hat})
f17_acc_d6hat = calculate_accuracy(model_f17, d6hat_loader)
print({f17_acc_d6hat})
f17_acc_d7hat = calculate_accuracy(model_f17, d7hat_loader)
print({f17_acc_d7hat})
f17_acc_d8hat = calculate_accuracy(model_f17, d8hat_loader)
print({f17_acc_d8hat})
f17_acc_d9hat = calculate_accuracy(model_f17, d9hat_loader)
print({f17_acc_d9hat})
f17_acc_d10hat = calculate_accuracy(model_f17, d10hat_loader)
print({f17_acc_d10hat})
f17_acc_d11hat = calculate_accuracy(model_f17, d11hat_loader)
print({f17_acc_d11hat})
f17_acc_d12hat = calculate_accuracy(model_f17, d12hat_loader)
print({f17_acc_d12hat})
f17_acc_d13hat = calculate_accuracy(model_f17, d13hat_loader)
print({f17_acc_d13hat})
f17_acc_d14hat = calculate_accuracy(model_f17, d14hat_loader)
print({f17_acc_d14hat})
f17_acc_d15hat = calculate_accuracy(model_f17, d15hat_loader)
print({f17_acc_d15hat})
f17_acc_d16hat = calculate_accuracy(model_f17, d16hat_loader)
print({f17_acc_d16hat})
f17_acc_d17hat = calculate_accuracy(model_f17, d17hat_loader)
print({f17_acc_d17hat})
accuracy_matrix.append([f17_acc_d1hat, f17_acc_d2hat, f17_acc_d3hat, f17_acc_d4hat, f17_acc_d5hat, f17_acc_d6hat, f17_acc_d7hat, f17_acc_d8hat, f17_acc_d9hat, f17_acc_d10hat, f17_acc_d11hat, f17_acc_d12hat, f17_acc_d13hat, f17_acc_d14hat, f17_acc_d15hat, f17_acc_d16hat, f17_acc_d17hat,None,None,None])




# Train on Unlabeled Data (D18) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f18_weights.pth'):
    model_f18 = copy.deepcopy(model)
    model_f18.load_state_dict(torch.load('/kaggle/input/weight-files/model_f18_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f18 = self_train_update_with_t2pl(model_f17, d18_data, 18)
    
print("started_accuracy_computation")
f18_acc_d1hat = calculate_accuracy(model_f18, d1hat_loader)
print({f18_acc_d1hat})
f18_acc_d2hat = calculate_accuracy(model_f18, d2hat_loader)
print({f18_acc_d2hat})
f18_acc_d3hat = calculate_accuracy(model_f18, d3hat_loader)
print({f18_acc_d3hat})
f18_acc_d4hat = calculate_accuracy(model_f18, d4hat_loader)
print({f18_acc_d4hat})
f18_acc_d5hat = calculate_accuracy(model_f18, d5hat_loader)
print({f18_acc_d5hat})
f18_acc_d6hat = calculate_accuracy(model_f18, d6hat_loader)
print({f18_acc_d6hat})
f18_acc_d7hat = calculate_accuracy(model_f18, d7hat_loader)
print({f18_acc_d7hat})
f18_acc_d8hat = calculate_accuracy(model_f18, d8hat_loader)
print({f18_acc_d8hat})
f18_acc_d9hat = calculate_accuracy(model_f18, d9hat_loader)
print({f18_acc_d9hat})
f18_acc_d10hat = calculate_accuracy(model_f18, d10hat_loader)
print({f18_acc_d10hat})
f18_acc_d11hat = calculate_accuracy(model_f18, d11hat_loader)
print({f18_acc_d11hat})
f18_acc_d12hat = calculate_accuracy(model_f18, d12hat_loader)
print({f18_acc_d12hat})
f18_acc_d13hat = calculate_accuracy(model_f18, d13hat_loader)
print({f18_acc_d13hat})
f18_acc_d14hat = calculate_accuracy(model_f18, d14hat_loader)
print({f18_acc_d14hat})
f18_acc_d15hat = calculate_accuracy(model_f18, d15hat_loader)
print({f18_acc_d15hat})
f18_acc_d16hat = calculate_accuracy(model_f18, d16hat_loader)
print({f18_acc_d16hat})
f18_acc_d17hat = calculate_accuracy(model_f18, d17hat_loader)
print({f18_acc_d17hat})
f18_acc_d18hat = calculate_accuracy(model_f18, d18hat_loader)
print({f18_acc_d18hat})
accuracy_matrix.append([f18_acc_d1hat, f18_acc_d2hat, f18_acc_d3hat, f18_acc_d4hat, f18_acc_d5hat, f18_acc_d6hat, f18_acc_d7hat, f18_acc_d8hat, f18_acc_d9hat, f18_acc_d10hat, f18_acc_d11hat, f18_acc_d12hat, f18_acc_d13hat, f18_acc_d14hat, f18_acc_d15hat, f18_acc_d16hat, f18_acc_d17hat, f18_acc_d18hat,None,None])



# Train on Unlabeled Data (D19) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f19_weights.pth'):
    model_f19 = copy.deepcopy(model)
    model_f19.load_state_dict(torch.load('/kaggle/input/weight-files/model_f19_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f19 = self_train_update_with_t2pl(model_f18, d19_data, 19)
    
print("started_accuracy_computation")
f19_acc_d1hat = calculate_accuracy(model_f19, d1hat_loader)
print({f19_acc_d1hat})
f19_acc_d2hat = calculate_accuracy(model_f19, d2hat_loader)
print({f19_acc_d2hat})
f19_acc_d3hat = calculate_accuracy(model_f19, d3hat_loader)
print({f19_acc_d3hat})
f19_acc_d4hat = calculate_accuracy(model_f19, d4hat_loader)
print({f19_acc_d4hat})
f19_acc_d5hat = calculate_accuracy(model_f19, d5hat_loader)
print({f19_acc_d5hat})
f19_acc_d6hat = calculate_accuracy(model_f19, d6hat_loader)
print({f19_acc_d6hat})
f19_acc_d7hat = calculate_accuracy(model_f19, d7hat_loader)
print({f19_acc_d7hat})
f19_acc_d8hat = calculate_accuracy(model_f19, d8hat_loader)
print({f19_acc_d8hat})
f19_acc_d9hat = calculate_accuracy(model_f19, d9hat_loader)
print({f19_acc_d9hat})
f19_acc_d10hat = calculate_accuracy(model_f19, d10hat_loader)
print({f19_acc_d10hat})
f19_acc_d11hat = calculate_accuracy(model_f19, d11hat_loader)
print({f19_acc_d11hat})
f19_acc_d12hat = calculate_accuracy(model_f19, d12hat_loader)
print({f19_acc_d12hat})
f19_acc_d13hat = calculate_accuracy(model_f19, d13hat_loader)
print({f19_acc_d13hat})
f19_acc_d14hat = calculate_accuracy(model_f19, d14hat_loader)
print({f19_acc_d14hat})
f19_acc_d15hat = calculate_accuracy(model_f19, d15hat_loader)
print({f19_acc_d15hat})
f19_acc_d16hat = calculate_accuracy(model_f19, d16hat_loader)
print({f19_acc_d16hat})
f19_acc_d17hat = calculate_accuracy(model_f19, d17hat_loader)
print({f19_acc_d17hat})
f19_acc_d18hat = calculate_accuracy(model_f19, d18hat_loader)
print({f19_acc_d18hat})
f19_acc_d19hat = calculate_accuracy(model_f19, d19hat_loader)
print({f19_acc_d19hat})
accuracy_matrix.append([f19_acc_d1hat, f19_acc_d2hat, f19_acc_d3hat, f19_acc_d4hat, f19_acc_d5hat, f19_acc_d6hat, f19_acc_d7hat, f19_acc_d8hat, f19_acc_d9hat, f19_acc_d10hat, f19_acc_d11hat, f19_acc_d12hat, f19_acc_d13hat, f19_acc_d14hat, f19_acc_d15hat, f19_acc_d16hat, f19_acc_d17hat, f19_acc_d18hat, f19_acc_d19hat,None])



# Train on Unlabeled Data (D20) using self-training and distillation
if os.path.exists('/kaggle/input/weight-files/model_f20_weights.pth'):
    model_f20 = copy.deepcopy(model)
    model_f20.load_state_dict(torch.load('/kaggle/input/weight-files/model_f20_weights.pth'))
    print("Loaded self-trained model.")
else:
    model_f20 = self_train_update_with_t2pl(model_f19, d20_data, 20)
    
print("started_accuracy_computation")
f20_acc_d1hat = calculate_accuracy(model_f20, d1hat_loader)
print({f20_acc_d1hat})
f20_acc_d2hat = calculate_accuracy(model_f20, d2hat_loader)
print({f20_acc_d2hat})
f20_acc_d3hat = calculate_accuracy(model_f20, d3hat_loader)
print({f20_acc_d3hat})
f20_acc_d4hat = calculate_accuracy(model_f20, d4hat_loader)
print({f20_acc_d4hat})
f20_acc_d5hat = calculate_accuracy(model_f20, d5hat_loader)
print({f20_acc_d5hat})
f20_acc_d6hat = calculate_accuracy(model_f20, d6hat_loader)
print({f20_acc_d6hat})
f20_acc_d7hat = calculate_accuracy(model_f20, d7hat_loader)
print({f20_acc_d7hat})
f20_acc_d8hat = calculate_accuracy(model_f20, d8hat_loader)
print({f20_acc_d8hat})
f20_acc_d9hat = calculate_accuracy(model_f20, d9hat_loader)
print({f20_acc_d9hat})
f20_acc_d10hat = calculate_accuracy(model_f20, d10hat_loader)
print({f20_acc_d10hat})
f20_acc_d11hat = calculate_accuracy(model_f20, d11hat_loader)
print({f20_acc_d11hat})
f20_acc_d12hat = calculate_accuracy(model_f20, d12hat_loader)
print({f20_acc_d12hat})
f20_acc_d13hat = calculate_accuracy(model_f20, d13hat_loader)
print({f20_acc_d13hat})
f20_acc_d14hat = calculate_accuracy(model_f20, d14hat_loader)
print({f20_acc_d14hat})
f20_acc_d15hat = calculate_accuracy(model_f20, d15hat_loader)
print({f20_acc_d15hat})
f20_acc_d16hat = calculate_accuracy(model_f20, d16hat_loader)
print({f20_acc_d16hat})
f20_acc_d17hat = calculate_accuracy(model_f20, d17hat_loader)
print({f20_acc_d17hat})
f20_acc_d18hat = calculate_accuracy(model_f20, d18hat_loader)
print({f20_acc_d18hat})
f20_acc_d19hat = calculate_accuracy(model_f20, d19hat_loader)
print({f20_acc_d19hat})
f20_acc_d20hat = calculate_accuracy(model_f20, d20hat_loader)
print({f20_acc_d20hat})
accuracy_matrix.append([f20_acc_d1hat, f20_acc_d2hat, f20_acc_d3hat, f20_acc_d4hat, f20_acc_d5hat, f20_acc_d6hat, f20_acc_d7hat, f20_acc_d8hat, f20_acc_d9hat, f20_acc_d10hat, f20_acc_d11hat, f20_acc_d12hat, f20_acc_d13hat, f20_acc_d14hat, f20_acc_d15hat, f20_acc_d16hat, f20_acc_d17hat, f20_acc_d18hat, f20_acc_d19hat, f20_acc_d20hat])

accuracy_matrix


{'reached_path_to_datasets'}


/tmp/ipykernel_30/3106921013.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_dict = torch.load(pth_path)
/opt/conda/lib/python3.10/site-packages/torchvision/models

{'started'}


/tmp/ipykernel_30/3106921013.py:485: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_f10.load_state_dict(torch.load('/kaggle/input/d10-weights/model_f10_weights.pth'))


Loaded self-trained model. for model 10
Epoch 1/15
Epoch 0 has a loss of 0.4599204659461975
Epoch 0 has a loss of 0.8011131286621094
Epoch 0 has a loss of 0.5198723673820496
Epoch 0 has a loss of 0.4204876720905304
Epoch 0 has a loss of 0.49412858486175537
Epoch 0 has a loss of 0.4465569257736206
Epoch 0 has a loss of 0.3646828234195709
Epoch 0 has a loss of 0.40532320737838745
Epoch 0 has a loss of 0.2633451223373413
Epoch 0 has a loss of 0.3439815938472748
Epoch 0 has a loss of 0.3762035667896271
Epoch 0 has a loss of 0.18445393443107605
Epoch 1, Loss: 0.4290723604361216
Epoch 2/15
Epoch 1 has a loss of 0.31932738423347473
Epoch 1 has a loss of 0.21292632818222046
Epoch 1 has a loss of 0.1085902601480484
Epoch 1 has a loss of 0.3186531066894531
Epoch 1 has a loss of 0.2185685783624649
Epoch 1 has a loss of 0.3829090893268585
Epoch 1 has a loss of 0.32523059844970703
Epoch 1 has a loss of 0.3222225308418274
Epoch 1 has a loss of 0.4065302610397339
Epoch 1 has a loss of 0.2006652057170

[[91.28,
  90.52,
  90.4,
  90.56,
  90.36,
  90.76,
  90.44,
  91.64,
  90.72,
  90.92,
  88.96,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [87.84,
  87.48,
  86.48,
  86.32,
  87.0,
  87.32,
  86.16,
  88.08,
  86.52,
  88.0,
  75.64,
  70.88,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [90.08,
  90.44,
  89.4,
  89.0,
  89.96,
  89.68,
  89.12,
  90.48,
  89.8,
  90.0,
  80.8,
  65.72,
  81.64,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [89.52,
  89.52,
  88.32,
  87.84,
  88.6,
  89.12,
  89.04,
  89.08,
  88.96,
  88.36,
  83.0,
  63.88,
  78.88,
  88.84,
  None,
  None,
  None,
  None,
  None,
  None],
 [90.24,
  90.32,
  88.8,
  89.0,
  89.68,
  89.6,
  89.16,
  90.2,
  89.68,
  89.76,
  83.72,
  60.12,
  78.56,
  88.4,
  89.6,
  None,
  None,
  None,
  None,
  None],
 [90.36,
  90.2,
  89.8,
  88.68,
  90.2,
  89.52,
  89.0,
  90.48,
  89.36,
  90.28,
  80.04,
  65.32,
  81.12,
  88.24,
  90.0,
  81.24,
  None,

Rand Mix using Ada In 